In [11]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import networkx.algorithms.community as nx_comm
import copy
import community

def FindGroup(groups, i):
    for index in range(len(groups)):
        if i in groups[index]:
            return index

def MovingNodes(G,Groups):
    groups = copy.deepcopy(Groups)
    for i in G.nodes:
        modularity = nx_comm.modularity(G, groups)
        index = FindGroup(groups, i)
        groups[index].remove(i)
        possibilities = []
#         for j in range(len(groups)):
#             print(j)
#             Groups = copy.deepcopy(groups)
#             Groups[j] += [i]
#             possibilities.append(Groups)
        for j in G.neighbors(i):
            Groups = copy.deepcopy(groups)
            index = FindGroup(Groups, j)
            Groups[index] += [i]
            possibilities.append(Groups)
            
        groups = max(possibilities, key=lambda x: nx_comm.modularity(G,x))
        if [] in groups:
            groups.remove([])
    return groups

def MovingGroups(G, Groups):
    modularity = nx_comm.modularity(G, Groups)
    for i in range(len(Groups)):
        groups = copy.deepcopy(Groups)
        g = groups[i]
        groups.remove(g)
        for j in range(len(groups)):
            new_group = copy.deepcopy(groups)
            new_group[j] += g
            if nx_comm.modularity(G, new_group) > modularity:
                return new_group
    return Groups
        
               
        
def LouvianMethod(G):
    # G is the graph we are interested in
    # n is the number of communities we want
    groups = []
    for i in G.nodes:
        groups.append([i])
    # Gropus = groups.copy()
    i = 0
    Finished = False
    while not Finished:
        i+=1
        new_groups = MovingNodes(G, groups)
        if new_groups == groups:
            Finished = True
        groups = new_groups
        #print(groups)
    Finished = False
    i = 0
    while not Finished:
        i+=1
        new_groups = MovingGroups(G,groups)
        if new_groups == groups:
            Finished = True
        groups = new_groups
        #print(groups)
    return groups
        

    
    



In [12]:


file = open("karate/zachary.dat")
f = file.readlines()

karate = nx.Graph()
karate.add_nodes_from(list(range(34)))

for i in range(34):
    line = f[i+7].split()
    for j in range(34):
        if line[j] == '1':
            karate.add_edge(i,j)
# fig = plt.figure(figsize=(15, 15)) 
# nx.draw(karate,node_size = 200,edge_color='r')
print(LouvianMethod(karate))


[[5, 6, 16, 4, 10], [0, 1, 11, 17, 19, 21, 2, 3, 7, 9, 12, 13], [23, 24, 25, 27, 28, 31], [8, 14, 15, 18, 20, 22, 26, 29, 30, 32, 33]]


In [6]:
test = nx.Graph()
test.add_edges_from([[1,2],[3,4],[5,7],[3,5],[3,6]])
print(list(test.neighbors(3)))
print(nx_comm.modularity(test,[[1],[2],[3],[4],[5],[6],[7]]))
print(nx_comm.modularity(test,[[1,3],[2],[4],[5],[6],[7]]))
print(nx_comm.modularity(test,[[1,2],[3],[4],[5],[6],[7]]))
LouvianMethod2(test)
#nx.draw(test)
# print(nx_comm.modularity(test,[[1],[2],[3],[4],[5],[6],[7]]))
# print(MovingNodes(test,[[1],[2],[3],[4],[5],[6],[7]]))
# print(LouvianMethod(test))
# print(LouvianMethod(test,2))
# print(nx_comm.modularity(test,[[1,2,3],[4,5,6,7]]))
# print(nx_comm.modularity(test,[[1,2,3,4],[5,6,7]]))
# print(nx_comm.modularity(test,[[1],[2,3,4,5,6,7]]))
# print(nx_comm.modularity(test,[[1,2,3,4,5,6,7]]))
# print(nx_comm.modularity(test,[[1,2],[3,4,5,6,7],[]]))

# print([1,2] in test.edges)
# print((1,2) in test)


[4, 5, 6]
-0.18000000000000005
-0.24000000000000005
5.551115123125783e-18
<built-in function round>
<built-in function round>
<built-in function round>


[[2, 1], [6, 7, 5, 4, 3]]

In [79]:
Mips = open("ppi/MIPSFirstLevel.anno3","r")
l = Mips.readline()

labels = {}
while l:
    line = l.split()
    if len(line) > 1:
        labels[line[0]] = set(line[1:])
    else:
        labels[line[0]] = set()
    
    l = Mips.readline()


ppi = nx.Graph()

file = open("ppi/confidence.ppi","r")
f = file.readline()


while f:
    line = f.split()
    ppi.add_edge(line[0],line[1])
    f = file.readline()
file.close()

partition = community.best_partition(ppi)

group_size  = max(partition.values()) + 1
groups = [[] for i in range(group_size)]
for i, j in partition.items():
    groups[j].append(i)

def score(group, labels):
    score = 0
    for i in group:
        for j in group:
            if i != j:
                l = labels[i].intersection(labels[j])
                if len(l) > 0:
                    score += 1
    score = score/(2*len(group) * (len(group)-1))
                   
    return score

scores = [0] * (group_size+1)
for i in range(group_size):
    scores[i] = score(groups[i],labels)

p3 = open("p3.txt","w")
for i in range(group_size):
    p3.write("cluster" + str(i+1) + ":\n")
    p3.write("score:" + str(scores[i]) + "\n")
    p3.write(str(groups[i]))
    p3.write('\n')
    p3.write('\n')
    
p3.close()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
